In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('data_1000_samples_clean.xlsx')
df

,Time,Temp,Weather,Wind,Humidity,Barometer,Visibility,Day,Month,Year
0,12:00 SA,22,Passing clouds.,4,88,1016,100,1,3,2022
1,12:30 SA,22,Passing clouds.,4,88,1015,100,1,3,2022
2,1:00 SA,22,Passing clouds.,4,83,1015,100,1,3,2022
3,1:30 SA,21,Passing clouds.,4,88,1015,8,1,3,2022
4,2:00 SA,21,Passing clouds.,2,88,1015,8,1,3,2022
...,...,...,...,...,...,...,...,...,...,...
996,5:00 CH,28,Passing clouds.,19,79,1009,100,22,3,2022
997,5:30 CH,28,Passing clouds.,13,79,1009,100,22,3,2022
998,6:00 CH,27,Passing clouds.,11,84,1009,100,22,3,2022
999,6:30 CH,27,Passing clouds.,11,84,1009,100,22,3,2022


In [3]:
train_df = df[['Barometer']][200:248]
val_df = df[['Barometer']][248:296]
test_df = df[['Barometer']][248:296]

In [4]:
# train_max = train_df.max()
# val_max = val_df.max()
# test_max = test_df.max()
# max_ = 0
# if train_max.Barometer > val_max.Barometer:
#     max_ = train_max
# else:
#     max_ = val_max
    
# train_df = train_df/ max_
# val_df = val_df/ max_
# test_df = test_df/ max_

### Feature Engineering: Window Features

In [5]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift,
                 train_df, val_df, test_df,
                 label_columns=None):

        self.train_df = train_df
        self.val_df = val_df
        self.test_df = test_df

        self.label_columns = label_columns # = ['Temp','Wind','Humidity','Barometer']
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in enumerate(label_columns)} #chỉ số label
        #{'Temp': 0, 'Wind': 1, 'Humidity': 2, 'Barometer': 3}
        
        self.column_indices = {name: i for i, name in enumerate(train_df.columns)} 
        #{'Temp': 0, 'Wind': 1, 'Humidity': 2, 'Barometer': 3}

        self.input_width = input_width # = 24
        self.label_width = label_width # = 24
        self.shift = shift # = 24

        self.total_window_size = input_width + shift # = 48

        self.input_slice = slice(0, input_width) # slice(0, 24)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice] # np.arange(48)[slice(0, 24)]

        self.label_start = self.total_window_size - self.label_width # = 24
        self.labels_slice = slice(self.label_start, None) # slice(24, 48)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice] # np.arange(48)[slice(24, 48)]

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'
        ])

    def train(self):
        return self.make_dataset(self.train_df)

    def val(self):
        return self.make_dataset(self.val_df)

    def test(self):
        return self.make_dataset(self.test_df)

In [6]:
# Hàm chia input và output
def split_window(self, features):
    #Chia dữ liệu thành 2 inputs và labels
    inputs = features[:, self.input_slice, :] #input_slice = slice(0, 24)
    labels = features[:, self.labels_slice, :] #labels_slice = slice(24, 48)
    
    #Lấy dữ liệu dựa trên label
    if self.label_columns is not None:
        # label_columns = ['Temp','Wind','Humidity','Barometer']
        # column_indices[Temp] = 0 ; column_indices[Wind] = 1;... 
        labels = tf.stack( [labels[:, :, self.column_indices[name]] for name in self.label_columns], axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.
    inputs.set_shape([None, self.input_width, None])
    labels.set_shape([None, self.label_width, None])

    return inputs, labels

WindowGenerator.split_window = split_window


In [7]:

def make_dataset(self, data):
    data = np.array(data, dtype=np.float32) # chuyển dataframe về numpy array
    ds = tf.keras.utils.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=False,
      batch_size=32,)
    ds = ds.map(self.split_window)

    return ds

WindowGenerator.make_dataset = make_dataset

In [8]:
@property
def train(self):
    return self.make_dataset(self.train_df)

@property
def val(self):
    return self.make_dataset(self.val_df)

@property
def test(self):
    return self.make_dataset(self.test_df)

@property
def example(self):
    """Get and cache an example batch of `inputs, labels` for plotting."""
    result = getattr(self, '_example', None)
    if result is None:
        # No example batch was found, so get one from the `.train` dataset
        result = next(iter(self.test))
        # And cache it for next time
        self._example = result
    return result

In [9]:
WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

In [10]:
OUT_STEPS = 24
multi_window = WindowGenerator(input_width=24,
                               label_width=OUT_STEPS,
                               shift=OUT_STEPS,
                              train_df = train_df,
                              val_df = val_df,
                              test_df = test_df,
                              label_columns = ['Barometer'])

In [11]:
multi_window.__repr__()

"Total window size: 48\nInput indices: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]\nLabel indices: [24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]\nLabel column name(s): ['Barometer']"

## Training

In [12]:
import tensorflow as tf
from sklearn.metrics import r2_score
MAX_EPOCHS = 200

def mean_2(y_true, y_pred):
    MAE = tf.reduce_mean(tf.abs(y_true - y_pred))

    y_sub = tf.subtract(y_true[:, 1:, :], y_true[:, :-1, :])
    y_hat_sub = tf.subtract(y_pred[:, 1:, :], y_pred[:, :-1, :])

    n = y_sub.shape[1] + y_sub.shape[2]
    
    A = y_sub*y_hat_sub + 1
    B = tf.sqrt(tf.square(y_sub) + 1)*tf.sqrt(tf.square(y_hat_sub) + 1)
    
    alpha = 1
    
    vector_similarity = tf.divide(A,B)
    sum_VS = tf.reduce_mean(vector_similarity)
    sum_VS = sum_VS*alpha
    
    MSE_2 = MAE + tf.math.reduce_std(tf.abs(y_true - y_pred))/MAE - sum_VS 
    
    return MSE_2

def CV(y_true, y_pred):
    MAE = tf.reduce_mean(tf.abs(y_true - y_pred))
    MSE_2 = tf.math.reduce_std(tf.abs(y_true - y_pred))/MAE 
    
    return MSE_2

def compile_and_fit(model, window, patience=2):
    model.compile(loss=tf.losses.MeanSquaredError(),
                  optimizer=tf.optimizers.Adam(),
                  metrics=[CV,mean_2,tf.metrics.MeanAbsoluteError()])
    history = model.fit(window.train, epochs=MAX_EPOCHS,
                        validation_data=window.val)
    return history


In [13]:
num_features = train_df.shape[1]
CONV_WIDTH = 3
multi_conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -3:, :]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
    # Shape => [batch, 1,  out_steps*features]
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features]
    tf.keras.layers.Reshape([OUT_STEPS, num_features])
])

In [14]:
history = compile_and_fit(multi_conv_model, multi_window)

Epoch 1/200
1/1 [==============================] - 1s 1s/step - loss: 1023302.0000 - CV: 0.0011 - mean_2: 1010.6608 - mean_absolute_error: 1011.5833 - val_loss: 1008748.5000 - val_CV: 0.0014 - val_mean_2: 1003.4415 - val_mean_absolute_error: 1004.3637
Epoch 2/200
1/1 [==============================] - 0s 51ms/step - loss: 1003417.5625 - CV: 0.0011 - mean_2: 1000.7843 - mean_absolute_error: 1001.7068 - val_loss: 987778.3125 - val_CV: 0.0014 - val_mean_2: 992.9471 - val_mean_absolute_error: 993.8693
Epoch 3/200
1/1 [==============================] - 0s 50ms/step - loss: 982523.7500 - CV: 0.0011 - mean_2: 990.3004 - mean_absolute_error: 991.2228 - val_loss: 965616.0625 - val_CV: 0.0014 - val_mean_2: 981.7346 - val_mean_absolute_error: 982.6567
Epoch 4/200
1/1 [==============================] - 0s 65ms/step - loss: 960442.5625 - CV: 0.0011 - mean_2: 979.0987 - mean_absolute_error: 980.0212 - val_loss: 942228.6875 - val_CV: 0.0015 - val_mean_2: 969.7615 - val_mean_absolute_error: 970.6836
E

Epoch 34/200
1/1 [==============================] - 0s 64ms/step - loss: 51273.5898 - CV: 0.0044 - mean_2: 225.5004 - mean_absolute_error: 226.4346 - val_loss: 37306.5664 - val_CV: 0.0069 - val_mean_2: 192.2228 - val_mean_absolute_error: 193.1445
Epoch 35/200
1/1 [==============================] - 0s 73ms/step - loss: 36592.0898 - CV: 0.0051 - mean_2: 190.3522 - mean_absolute_error: 191.2881 - val_loss: 25113.8594 - val_CV: 0.0083 - val_mean_2: 157.5472 - val_mean_absolute_error: 158.4680
Epoch 36/200
1/1 [==============================] - 0s 74ms/step - loss: 24538.8496 - CV: 0.0061 - mean_2: 155.7081 - mean_absolute_error: 156.6459 - val_loss: 15514.0977 - val_CV: 0.0105 - val_mean_2: 123.6294 - val_mean_absolute_error: 124.5488
Epoch 37/200
1/1 [==============================] - 0s 48ms/step - loss: 15070.9092 - CV: 0.0076 - mean_2: 121.8206 - mean_absolute_error: 122.7601 - val_loss: 8402.8213 - val_CV: 0.0140 - val_mean_2: 90.7415 - val_mean_absolute_error: 91.6579
Epoch 38/200
1/

Epoch 68/200
1/1 [==============================] - 0s 58ms/step - loss: 170.8335 - CV: 0.0158 - mean_2: 12.0855 - mean_absolute_error: 13.0687 - val_loss: 439.2715 - val_CV: 0.0411 - val_mean_2: 20.0958 - val_mean_absolute_error: 20.9412
Epoch 69/200
1/1 [==============================] - 0s 58ms/step - loss: 370.7897 - CV: 0.0108 - mean_2: 18.2666 - mean_absolute_error: 19.2548 - val_loss: 687.4589 - val_CV: 0.0328 - val_mean_2: 25.3517 - val_mean_absolute_error: 26.2053
Epoch 70/200
1/1 [==============================] - 0s 66ms/step - loss: 600.9665 - CV: 0.0085 - mean_2: 23.5232 - mean_absolute_error: 24.5137 - val_loss: 933.4466 - val_CV: 0.0281 - val_mean_2: 29.6818 - val_mean_absolute_error: 30.5403
Epoch 71/200
1/1 [==============================] - 0s 69ms/step - loss: 832.0396 - CV: 0.0071 - mean_2: 27.8525 - mean_absolute_error: 28.8444 - val_loss: 1154.0095 - val_CV: 0.0253 - val_mean_2: 33.0983 - val_mean_absolute_error: 33.9599
Epoch 72/200
1/1 [=========================

1/1 [==============================] - 0s 61ms/step - loss: 109.1705 - CV: 0.0037 - mean_2: 9.4522 - mean_absolute_error: 10.4484 - val_loss: 66.1553 - val_CV: 0.1055 - val_mean_2: 7.2939 - val_mean_absolute_error: 8.0887
Epoch 103/200
1/1 [==============================] - 0s 58ms/step - loss: 94.9938 - CV: 0.0040 - mean_2: 8.7504 - mean_absolute_error: 9.7464 - val_loss: 53.1409 - val_CV: 0.1179 - val_mean_2: 6.4573 - val_mean_absolute_error: 7.2396
Epoch 104/200
1/1 [==============================] - 0s 59ms/step - loss: 79.1798 - CV: 0.0043 - mean_2: 7.9026 - mean_absolute_error: 8.8982 - val_loss: 40.1181 - val_CV: 0.1360 - val_mean_2: 5.5119 - val_mean_absolute_error: 6.2761
Epoch 105/200
1/1 [==============================] - 0s 55ms/step - loss: 62.9754 - CV: 0.0047 - mean_2: 6.9403 - mean_absolute_error: 7.9356 - val_loss: 28.0780 - val_CV: 0.1631 - val_mean_2: 4.4928 - val_mean_absolute_error: 5.2297
Epoch 106/200
1/1 [==============================] - 0s 51ms/step - loss: 47

1/1 [==============================] - 0s 54ms/step - loss: 0.7978 - CV: 0.0066 - mean_2: -0.1002 - mean_absolute_error: 0.8932 - val_loss: 1.0977 - val_CV: 0.6010 - val_mean_2: 0.6012 - val_mean_absolute_error: 0.8980
Epoch 138/200
1/1 [==============================] - 0s 56ms/step - loss: 1.1086 - CV: 0.0053 - mean_2: 0.0582 - mean_absolute_error: 1.0529 - val_loss: 0.9671 - val_CV: 0.5578 - val_mean_2: 0.5189 - val_mean_absolute_error: 0.8588
Epoch 139/200
1/1 [==============================] - 0s 53ms/step - loss: 1.3701 - CV: 0.0045 - mean_2: 0.1749 - mean_absolute_error: 1.1705 - val_loss: 0.8972 - val_CV: 0.5401 - val_mean_2: 0.4757 - val_mean_absolute_error: 0.8334
Epoch 140/200
1/1 [==============================] - 0s 55ms/step - loss: 1.5549 - CV: 0.0039 - mean_2: 0.2509 - mean_absolute_error: 1.2470 - val_loss: 0.8673 - val_CV: 0.5355 - val_mean_2: 0.4587 - val_mean_absolute_error: 0.8210
Epoch 141/200
1/1 [==============================] - 0s 55ms/step - loss: 1.6492 - CV

1/1 [==============================] - 0s 59ms/step - loss: 0.0011 - CV: 0.0246 - mean_2: -0.9424 - mean_absolute_error: 0.0330 - val_loss: 3.4604 - val_CV: 0.5092 - val_mean_2: 1.2688 - val_mean_absolute_error: 1.6577
Epoch 173/200
1/1 [==============================] - 0s 60ms/step - loss: 1.5761e-04 - CV: 0.0656 - mean_2: -0.9219 - mean_absolute_error: 0.0125 - val_loss: 3.3269 - val_CV: 0.5066 - val_mean_2: 1.2356 - val_mean_absolute_error: 1.6271
Epoch 174/200
1/1 [==============================] - 0s 56ms/step - loss: 0.0028 - CV: 0.0135 - mean_2: -0.9332 - mean_absolute_error: 0.0533 - val_loss: 3.2140 - val_CV: 0.5046 - val_mean_2: 1.2070 - val_mean_absolute_error: 1.6006
Epoch 175/200
1/1 [==============================] - 0s 43ms/step - loss: 0.0079 - CV: 0.0071 - mean_2: -0.9043 - mean_absolute_error: 0.0886 - val_loss: 3.1224 - val_CV: 0.5030 - val_mean_2: 1.1835 - val_mean_absolute_error: 1.5786
Epoch 176/200
1/1 [==============================] - 0s 51ms/step - loss: 0.01

In [ ]:
multi_conv_model.summary()

## Trực quan dữ liệu sau khi Training

In [ ]:
def plot(self, model=None, plot_col='Temp', max_subplots=3):
    inputs, labels = self.example
    plt.figure(figsize=(19, 3))
    plot_col_index = self.column_indices[plot_col]
    max_n = min(max_subplots, len(inputs))
    for n in range(max_n):
        plt.subplot(max_n, 1, n+1)
        plt.ylabel(f'{plot_col}')
        plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                 label='Inputs', marker='.', zorder=-10)

        if self.label_columns:
            label_col_index = self.label_columns_indices.get(plot_col, None)
        else:
            label_col_index = plot_col_index

        if label_col_index is None:
            continue

        plt.scatter(self.label_indices, labels[n, :, label_col_index],
                    edgecolors='k', label='Labels', c='#2ca02c', s=64)
        if model is not None:
            predictions = model(inputs)
            plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                        marker='X', edgecolors='k', label='Predictions',
                        c='#ff7f0e', s=64)

        if n == 0:
            plt.legend()

    plt.xlabel('Time ')

WindowGenerator.plot = plot


### Độ chính xác cao hơn mô hình cũ

In [ ]:
multi_window.plot(multi_conv_model,plot_col='Barometer')

In [ ]:
# Vẽ đồ thị
# Chỉnh kích thước đồ thị
plt.plot(history.history['val_CV'], label='CV')
plt.plot(history.history['val_mean_2'], label='New Metric')
plt.plot(history.history['val_mean_absolute_error'], label='MAE')

# Đặt tên trục x và trục y
plt.xlabel('epochs')
plt.ylabel('loss')

# Thêm chú thích
plt.legend()
plt.figure(figsize=(8, 13))
# Hiển thị đồ thị
plt.show()

In [ ]:
inputs, labels = multi_window.example
predict = np.array(multi_conv_model(inputs)).reshape(24)
labels = np.array(labels).reshape(24)
labels.shape

In [ ]:
predict.shape

In [ ]:
from scipy.stats import ks_2samp
# Thực hiện kiểm định Kolmogorov-Smirnov
statistic, p_value = ks_2samp(predict, labels)

# In kết quả
print("Statistic:", statistic)
print("p-value:", p_value)

In [ ]:
# def mean_2(y_true, y_pred):
#     MAE = tf.reduce_mean(tf.abs(y_true - y_pred))

#     y_sub = tf.subtract(y_true[:, 1:, :], y_true[:, :-1, :])
#     y_hat_sub = tf.subtract(y_pred[:, 1:, :], y_pred[:, :-1, :])

#     n = y_sub.shape[1] + y_sub.shape[2]
    
#     A = y_sub*y_hat_sub + 1
#     B = tf.sqrt(tf.square(y_sub) + 1)*tf.sqrt(tf.square(y_hat_sub) + 1)
    
#     alpha = 1
    
#     vector_similarity = tf.divide(A,B)
#     sum_VS = tf.reduce_mean(vector_similarity)
#     sum_VS = sum_VS*alpha

    
#     MSE_2 = MAE - sum_VS
    
#     return sum_VS